In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
pip install flaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 205 kB 4.9 MB/s 
     |████████████████████████████████| 2.0 MB 87.8 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [ ]:
import pandas as pd 
from sqlite3 import connect

conn = connect('/content/drive/MyDrive/geotesouro/data/data.db')
convenios = pd.read_sql('SELECT * FROM convenios', conn)

In [ ]:
agg_municipios = pd.read_sql('SELECT * FROM agg_municipios', conn)
#agg_municipios = agg_municipios[agg_municipios.columns[0:14]]

In [ ]:
convenios = convenios[convenios["TIPO ENTE CONVENENTE"]=="Municipal"]
convenios = convenios[convenios['CÓDIGO SIAFI MUNICÍPIO']>0]
convenios = convenios[["NÚMERO CONVÊNIO", "CÓDIGO SIAFI MUNICÍPIO", "DATA PUBLICAÇÃO", "OBJETO DO CONVÊNIO", 'NOME ÓRGÃO SUPERIOR', "VALOR CONVÊNIO"]]
convenios["VALOR CONVÊNIO"] = convenios["VALOR CONVÊNIO"].replace(",", ".", regex=True).astype(float)
convenios["DATA PUBLICAÇÃO"] = pd.to_datetime(convenios["DATA PUBLICAÇÃO"])
convenios = convenios[convenios["DATA PUBLICAÇÃO"] > '2010-01-01']
convenios = convenios[convenios["DATA PUBLICAÇÃO"] < '2022-01-01']
convenios["ANO"] = convenios["DATA PUBLICAÇÃO"].dt.year

### **Modelo - Previsão Covênio Geral**

In [ ]:
convenios_ = convenios[["CÓDIGO SIAFI MUNICÍPIO", "ANO", "VALOR CONVÊNIO"]].groupby(['CÓDIGO SIAFI MUNICÍPIO', 'ANO']).sum().reset_index()
convenios_ = convenios_.rename(columns = {"CÓDIGO SIAFI MUNICÍPIO": "siafi_id", "VALOR CONVÊNIO": "valor_convenio_anual", "ANO": "ano"})

In [ ]:
list_cols = agg_municipios.columns.to_list()
list_cols.append("target")
list_cols.append("ano")
dataset = pd.DataFrame(columns = list_cols)

for a in convenios_["ano"].unique():
  target = convenios_[convenios_["ano"]==a]
  target["target"] = 1
  target = pd.merge(agg_municipios, target[["siafi_id", "target"]], how = "left", on = "siafi_id")
  target["target"] = target["target"].fillna(0).astype(int)
  target["ano"] = a
  dataset = dataset.append(target, ignore_index=True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
X_train = dataset[dataset["ano"]!=2021].drop(["codigo_ibge", "nome", "siafi_id", "target"], 1)
y_train = dataset[dataset["ano"]!=2021]["target"]
X_test = dataset[dataset["ano"]==2021].drop(["codigo_ibge", "nome", "siafi_id", "target"], 1)
y_test = dataset[dataset["ano"]==2021]["target"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
pip install flaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### XGBoost

In [ ]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'roc_auc', 
    "estimator_list": ['xgboost'],  
    "task": 'classification',  
    "seed": 7654321,    
}
automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 08-30 21:37:28] {2565} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 08-30 21:37:28] {2567} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 08-30 21:37:28] {2570} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 08-30 21:37:29] {2689} INFO - Minimizing error metric: 1-roc_auc
INFO:flaml.automl:Minimizing error metric: 1-roc_auc
[flaml.automl: 08-30 21:37:29] {2831} INFO - List of ML learners in AutoML Run: ['xgboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 08-30 21:37:29] {3133} INFO - iteration 0, current learner xgboost
INFO:flaml.automl:iteration 0, current learner xgboost
[flaml.automl: 08-30 21:37:31] {3267} INFO - Estimated sufficient time budget=106157s. Estimated necessary time budget=106s.
INFO:flaml.automl:Estimated sufficient time budget=106157s. Estimated necessary time budget=106s.
[fl

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: xgboost
Best hyperparmeter config: {'n_estimators': 155, 'max_leaves': 22, 'min_child_weight': 38.19348530611614, 'learning_rate': 0.23710347301788823, 'subsample': 0.6776066367365315, 'colsample_bylevel': 0.856947765772267, 'colsample_bytree': 0.8732202347444752, 'reg_alpha': 1.1437640566648772, 'reg_lambda': 234.69528545200882, 'FLAML_sample_size': 55143}
Best accuracy on validation data: 0.7848
Training duration of best run: 23.39 s


In [ ]:
automl.model.estimator

XGBClassifier(colsample_bylevel=0.856947765772267,
              colsample_bytree=0.8732202347444752, grow_policy='lossguide',
              learning_rate=0.23710347301788823, max_depth=0, max_leaves=22,
              min_child_weight=38.19348530611614, n_estimators=155, n_jobs=-1,
              reg_alpha=1.1437640566648772, reg_lambda=234.69528545200882,
              subsample=0.6776066367365315, tree_method='hist',
              use_label_encoder=False, verbosity=0)

In [ ]:
from flaml.ml import sklearn_metric_loss_score

y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

accuracy = 0.7795332136445242
roc_auc = 0.657188160636882
log_loss = 0.6335478883282628


In [ ]:
performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Convênios", "Área": "Geral", "Algoritmo": "XGBoost",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

In [ ]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/convenios/mdl_xgb_convenio_geral.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

#### LightGBM

In [ ]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'roc_auc', 
    "estimator_list": ['lgbm'],  
    "task": 'classification',  
    "seed": 7654321,    
}
automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 08-30 22:22:54] {2565} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 08-30 22:22:54] {2567} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 08-30 22:22:54] {2570} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 08-30 22:22:54] {2689} INFO - Minimizing error metric: 1-roc_auc
INFO:flaml.automl:Minimizing error metric: 1-roc_auc
[flaml.automl: 08-30 22:22:54] {2831} INFO - List of ML learners in AutoML Run: ['lgbm']
INFO:flaml.automl:List of ML learners in AutoML Run: ['lgbm']
[flaml.automl: 08-30 22:22:54] {3133} INFO - iteration 0, current learner lgbm
INFO:flaml.automl:iteration 0, current learner lgbm
[flaml.automl: 08-30 22:22:55] {3267} INFO - Estimated sufficient time budget=39530s. Estimated necessary time budget=40s.
INFO:flaml.automl:Estimated sufficient time budget=39530s. Estimated necessary time budget=40s.
[flaml.automl: 08-3

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 171, 'num_leaves': 4, 'min_child_samples': 36, 'learning_rate': 0.4536631114884281, 'log_max_bin': 8, 'colsample_bytree': 0.7670576821890048, 'reg_alpha': 0.011921948704584044, 'reg_lambda': 1.05110195634849, 'FLAML_sample_size': 55143}
Best accuracy on validation data: 0.7711
Training duration of best run: 13.1 s


In [ ]:
automl.model.estimator

LGBMClassifier(colsample_bytree=0.7670576821890048,
               learning_rate=0.4536631114884281, max_bin=255,
               min_child_samples=36, n_estimators=171, num_leaves=4,
               reg_alpha=0.011921948704584044, reg_lambda=1.05110195634849,
               verbose=-1)

In [ ]:
from flaml.ml import sklearn_metric_loss_score

y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

accuracy = 0.718491921005386
roc_auc = 0.6801161520606995
log_loss = 0.5423452526785923


In [ ]:
performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Convênios", "Área": "Geral", "Algoritmo": "LGBM",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

In [ ]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/convenios/mdl_lgbm_convenio_geral.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

#### CatBoost

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'roc_auc', 
    "estimator_list": ['catboost'],  
    "task": 'classification',  
    "seed": 7654321,    
}
automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 08-30 22:29:11] {2565} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 08-30 22:29:11] {2567} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 08-30 22:29:11] {2570} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 08-30 22:29:11] {2689} INFO - Minimizing error metric: 1-roc_auc
INFO:flaml.automl:Minimizing error metric: 1-roc_auc
[flaml.automl: 08-30 22:29:11] {2831} INFO - List of ML learners in AutoML Run: ['catboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['catboost']
[flaml.automl: 08-30 22:29:11] {3133} INFO - iteration 0, current learner catboost
INFO:flaml.automl:iteration 0, current learner catboost
[flaml.automl: 08-30 22:29:23] {3267} INFO - Estimated sufficient time budget=631933s. Estimated necessary time budget=632s.
INFO:flaml.automl:Estimated sufficient time budget=631933s. Estimated necessary time budget=632s.

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: catboost
Best hyperparmeter config: {'early_stopping_rounds': 10, 'learning_rate': 0.2, 'n_estimators': 42, 'FLAML_sample_size': 55143}
Best accuracy on validation data: 0.7895
Training duration of best run: 13.05 s


In [ ]:
automl.model.estimator

In [ ]:
from flaml.ml import sklearn_metric_loss_score

y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

accuracy = 0.7585278276481149
roc_auc = 0.7008788111574397
log_loss = 0.5008754585207681


In [ ]:
performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Convênios", "Área": "Geral", "Algoritmo": "CatBoost",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

In [ ]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/convenios/mdl_cb_convenio_geral.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

#### Ensemble 

In [ ]:
with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/convenios/mdl_xgb_convenio_geral.pkl', 'rb') as f:
    mdl_xgb_convenio_geral = pickle.load(f)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/convenios/mdl_lgbm_convenio_geral.pkl', 'rb') as f:
    mdl_lgbm_convenio_geral = pickle.load(f)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/convenios/mdl_cb_convenio_geral.pkl', 'rb') as f:
    mdl_cb_convenio_geral = pickle.load(f)   

In [ ]:
pred_train_ensem = pd.DataFrame()
pred_train_ensem[["xgb_proba_0", "xgb_proba_1"]] = mdl_xgb_convenio_geral.predict_proba(X_train)
pred_train_ensem[["lgbm_proba_0", "lgbm_proba_1"]] = mdl_lgbm_convenio_geral.predict_proba(X_train)
pred_train_ensem[["cb_proba_0", "cb_proba_1"]] = mdl_cb_convenio_geral.predict_proba(X_train)
pred_train_ensem["y_train"] = y_train.reset_index(drop=True)

In [ ]:
pred_test_ensem = pd.DataFrame()
pred_test_ensem[["xgb_proba_0", "xgb_proba_1"]] = mdl_xgb_convenio_geral.predict_proba(X_test)
pred_test_ensem[["lgbm_proba_0", "lgbm_proba_1"]] = mdl_lgbm_convenio_geral.predict_proba(X_test)
pred_test_ensem[["cb_proba_0", "cb_proba_1"]] = mdl_cb_convenio_geral.predict_proba(X_test)
pred_test_ensem["y_test"] = y_test.reset_index(drop=True)

In [ ]:
X_train_ensem = pred_train_ensem.drop(["y_train"], 1)
X_test_ensem = pred_test_ensem.drop(["y_test"], 1)
y_train_ensem = pred_train_ensem["y_train"]
y_test_ensem = pred_test_ensem["y_test"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [ ]:
from sklearn.linear_model import LogisticRegression
from flaml.ml import sklearn_metric_loss_score

clf = LogisticRegression().fit(X_train_ensem, y_train_ensem.astype(int))
y_pred = clf.predict(X_test_ensem)
y_pred_proba = clf.predict_proba(X_test_ensem)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

accuracy = 0.7666068222621185
roc_auc = 0.6628499453110817
log_loss = 0.5193115052427834


In [ ]:
performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Convênios", "Área": "Geral", "Algoritmo": "Ensemble",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

In [ ]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/transferencias/mdl_ensem_transf_geral.pkl', 'wb') as f:
    pickle.dump(clf, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
preds_convenios = pd.DataFrame()
preds_convenios = dataset[dataset["ano"]==2021][["codigo_ibge", "siafi_id", "nome"]].reset_index(drop=True)

In [ ]:
X_preds = X_test
X_preds["ano"] = "2022"

preds_convenios[["xgb_proba_0", "xgb_proba_1"]] = mdl_xgb_convenio_geral.predict_proba(X_preds)
preds_convenios[["lgbm_proba_0", "lgbm_proba_1"]] = mdl_lgbm_convenio_geral.predict_proba(X_preds)
preds_convenios[["cb_proba_0", "cb_proba_1"]] = mdl_cb_convenio_geral.predict_proba(X_preds)

In [ ]:
preds_convenios[["candido_pred_proba_0", "candido_pred_proba_1"]] = clf.predict_proba(preds_convenios.drop(["codigo_ibge", "siafi_id", "nome"], 1))
preds_convenios["candido_pred"] = clf.predict(preds_convenios.drop(["codigo_ibge", "siafi_id", "nome", "candido_pred_proba_0", "candido_pred_proba_1"], 1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [ ]:
preds_convenios.to_sql('preds_convenios', con=conn, if_exists='replace', index=False)

#### Feature Importance

In [ ]:
feature_importance = pd.DataFrame()
feature_importance["feature_names"] = X_preds.columns
feature_importance["feature_score"] = mdl_cb_convenio_geral.model.estimator.feature_importances_
feature_importance = feature_importance.sort_values(by=['feature_score'], ascending=False)
feature_importance

,feature_names,feature_score
58,va_2004,46.247568
1,longitude,9.382012
23,pop_2003,6.497878
2,capital,3.609486
74,va_industria_2006,2.866030
...,...,...
105,pib_2011,0.000000
40,pop_2020,0.000000
41,pop_2021,0.000000
42,pib_2002,0.000000


### **Modelo Residual - Órgão Superior (Ministério) Convênio**

In [ ]:
counts = convenios["NOME ÓRGÃO SUPERIOR"].value_counts()
counts_ministerios = pd.DataFrame(counts).reset_index().rename(columns = {"index": "ministerio", "NOME ÓRGÃO SUPERIOR": "count_ocorr"})
list_mins = counts_ministerios[counts_ministerios["count_ocorr"] < 1000]["ministerio"].to_list()
convenios["NOME ÓRGÃO SUPERIOR"] = ["Outros" if any(min in m for min in list_mins) else m for m in convenios["NOME ÓRGÃO SUPERIOR"]]
convenios_ = convenios[["CÓDIGO SIAFI MUNICÍPIO", "NOME ÓRGÃO SUPERIOR", "ANO", "VALOR CONVÊNIO"]].groupby(['CÓDIGO SIAFI MUNICÍPIO', "NOME ÓRGÃO SUPERIOR", 'ANO']).sum().reset_index()
convenios_ = convenios_.rename(columns = {"CÓDIGO SIAFI MUNICÍPIO": "siafi_id", "VALOR CONVÊNIO": "valor_convenio_anual", "ANO": "ano", "NOME ÓRGÃO SUPERIOR": "orgao_superior"})

In [ ]:
mins = convenios["NOME ÓRGÃO SUPERIOR"].unique()
dict_orgaos = {'Ministério do Desenvolvimento Regional': "min_desenv_reg",
               'Ministério do Turismo': 'min_turismo', 'Ministério da Ciência, Tecnologia, Inovações e Comunicações': "min_ciencia_tec_inov_comun",
               'Outros': "min_outros", 'Ministério da Mulher, Família e Direitos Humanos': "min_mulher_fam_dh",
               "Ministério da Saúde": 'min_saude', 'Ministério da Agricultura, Pecuária e Abastecimento': 'min_agri_pecu_abast',
               'Ministério da Justiça e Segurança Pública': "min_just_seg", "Ministério da Educação": "min_educ", 
               "Ministério da Cidadania": "min_cidad", "Ministério da Defesa": "min_def"}

models = ["xgboost", "lgbm", "catboost"]
mdls = {"xgboost": ["xgb", "XGBoost"], "lgbm": ["lgbm", "LGBM"], "catboost": ["cb", "CatBoost"]}

In [ ]:
from flaml import AutoML
from flaml.ml import sklearn_metric_loss_score
import pickle
from tqdm import tqdm 

for m in mins: 
  list_cols = agg_municipios.columns.to_list()
  list_cols.append("target")
  list_cols.append("ano")
  dataset = pd.DataFrame(columns = list_cols)

  for a in convenios_["ano"].unique():
    target = convenios_[(convenios_["orgao_superior"]==m) & (convenios_["ano"]==a)]
    target["target"] = 1
    target = pd.merge(agg_municipios, target[["siafi_id", "target"]], how = "left", on = "siafi_id")
    target["target"] = target["target"].fillna(0).astype(int)
    target["ano"] = a
    dataset = dataset.append(target, ignore_index=True)
  
  X_train = dataset[dataset["ano"]!=2021].drop(["codigo_ibge", "nome", "siafi_id", "target"], 1)
  y_train = dataset[dataset["ano"]!=2021]["target"]
  X_test = dataset[dataset["ano"]==2021].drop(["codigo_ibge", "nome", "siafi_id", "target"], 1)
  y_test = dataset[dataset["ano"]==2021]["target"]
  
  for p in models:

    automl = AutoML()
    settings = {
        "time_budget": 360,  
        "metric": 'roc_auc', 
        "estimator_list": [p],  
        "task": 'classification',  
        "seed": 7654321,    
    }
    automl.fit(X_train = X_train, y_train = y_train, **settings)

    y_pred = automl.predict(X_test)
    y_pred_proba = automl.predict_proba(X_test)[:,1]

    performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
    performance_mdl = performance_mdl.append({"Tema": "Convênios", "Área": dict_orgaos[m], "Algoritmo": mdls[p][1],
                                              "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                              "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                              "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

    performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

    with open(f'/content/drive/MyDrive/geotesouro/modelagem/models_save/convenios/mdl_{mdls[p][0]}_{dict_orgaos[m]}.pkl', 'wb') as f:
        pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
[flaml.automl: 08-31 12:06:40] {2565} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 08-31 12:06:40] {2567} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 08-31 1

#### Ensemble

In [ ]:
import pickle
from sklearn.linear_model import LogisticRegression
from flaml.ml import sklearn_metric_loss_score
import pandas as pd 
from tqdm import tqdm

preds_conv = pd.read_sql('select * from preds_convenios', con = conn)

counts = convenios["NOME ÓRGÃO SUPERIOR"].value_counts()
counts_ministerios = pd.DataFrame(counts).reset_index().rename(columns = {"index": "ministerio", "NOME ÓRGÃO SUPERIOR": "count_ocorr"})
list_mins = counts_ministerios[counts_ministerios["count_ocorr"] < 1000]["ministerio"].to_list()
convenios["NOME ÓRGÃO SUPERIOR"] = ["Outros" if any(min in m for min in list_mins) else m for m in convenios["NOME ÓRGÃO SUPERIOR"]]
convenios_ = convenios[["CÓDIGO SIAFI MUNICÍPIO", "NOME ÓRGÃO SUPERIOR", "ANO", "VALOR CONVÊNIO"]].groupby(['CÓDIGO SIAFI MUNICÍPIO', "NOME ÓRGÃO SUPERIOR", 'ANO']).sum().reset_index()
convenios_ = convenios_.rename(columns = {"CÓDIGO SIAFI MUNICÍPIO": "siafi_id", "VALOR CONVÊNIO": "valor_convenio_anual", "ANO": "ano", "NOME ÓRGÃO SUPERIOR": "orgao_superior"})

mins = convenios["NOME ÓRGÃO SUPERIOR"].unique()
dict_orgaos = {'Ministério do Desenvolvimento Regional': "min_desenv_reg",
               'Ministério do Turismo': 'min_turismo', 'Ministério da Ciência, Tecnologia, Inovações e Comunicações': "min_ciencia_tec_inov_comun",
               'Outros': "min_outros", 'Ministério da Mulher, Família e Direitos Humanos': "min_mulher_fam_dh",
               "Ministério da Saúde": 'min_saude', 'Ministério da Agricultura, Pecuária e Abastecimento': 'min_agri_pecu_abast',
               'Ministério da Justiça e Segurança Pública': "min_just_seg", "Ministério da Educação": "min_educ", 
               "Ministério da Cidadania": "min_cidad", "Ministério da Defesa": "min_def"}

for m in tqdm(mins): 
  list_cols = agg_municipios.columns.to_list()
  list_cols.append("target")
  list_cols.append("ano")
  dataset = pd.DataFrame(columns = list_cols)

  for a in convenios_["ano"].unique():
    target = convenios_[(convenios_["orgao_superior"]==m) & (convenios_["ano"]==a)]
    target["target"] = 1
    target = pd.merge(agg_municipios, target[["siafi_id", "target"]], how = "left", on = "siafi_id")
    target["target"] = target["target"].fillna(0).astype(int)
    target["ano"] = a
    dataset = dataset.append(target, ignore_index=True)
  
  X_train = dataset[dataset["ano"]!=2021].drop(["codigo_ibge", "nome", "siafi_id", "target"], 1)
  y_train = dataset[dataset["ano"]!=2021]["target"]
  X_test = dataset[dataset["ano"]==2021].drop(["codigo_ibge", "nome", "siafi_id", "target"], 1)
  y_test = dataset[dataset["ano"]==2021]["target"]

  with open(f'/content/drive/MyDrive/geotesouro/modelagem/models_save/convenios/mdl_xgb_{dict_orgaos[m]}.pkl', 'rb') as f:
      mdl_xgb_convenio_res = pickle.load(f)

  with open(f'/content/drive/MyDrive/geotesouro/modelagem/models_save/convenios/mdl_lgbm_{dict_orgaos[m]}.pkl', 'rb') as f:
      mdl_lgbm_convenio_res = pickle.load(f)

  with open(f'/content/drive/MyDrive/geotesouro/modelagem/models_save/convenios/mdl_cb_{dict_orgaos[m]}.pkl', 'rb') as f:
      mdl_cb_convenio_res = pickle.load(f)   

  pred_train_ensem = pd.DataFrame()
  pred_train_ensem[["xgb_proba_0", "xgb_proba_1"]] = mdl_xgb_convenio_res.predict_proba(X_train)
  pred_train_ensem[["lgbm_proba_0", "lgbm_proba_1"]] = mdl_lgbm_convenio_res.predict_proba(X_train)
  pred_train_ensem[["cb_proba_0", "cb_proba_1"]] = mdl_cb_convenio_res.predict_proba(X_train)
  pred_train_ensem["y_train"] = y_train.reset_index(drop=True)

  pred_test_ensem = pd.DataFrame()
  pred_test_ensem[["xgb_proba_0", "xgb_proba_1"]] = mdl_xgb_convenio_res.predict_proba(X_test)
  pred_test_ensem[["lgbm_proba_0", "lgbm_proba_1"]] = mdl_lgbm_convenio_res.predict_proba(X_test)
  pred_test_ensem[["cb_proba_0", "cb_proba_1"]] = mdl_cb_convenio_res.predict_proba(X_test)
  pred_test_ensem["y_test"] = y_test.reset_index(drop=True)

  X_train_ensem = pred_train_ensem.drop(["y_train"], 1)
  X_test_ensem = pred_test_ensem.drop(["y_test"], 1)
  y_train_ensem = pred_train_ensem["y_train"]
  y_test_ensem = pred_test_ensem["y_test"]

  clf = LogisticRegression().fit(X_train_ensem, y_train_ensem.astype(int))
  y_pred = clf.predict(X_test_ensem)
  y_pred_proba = clf.predict_proba(X_test_ensem)[:,1]
  print(f'accuracy - {dict_orgaos[m]}', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
  print(f'roc_auc - {dict_orgaos[m]}', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
  print(f'log_loss -  {dict_orgaos[m]}', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))


  performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
  performance_mdl = performance_mdl.append({"Tema": "Convênios", "Área": dict_orgaos[m], "Algoritmo": "Ensemble",
                                            "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                            "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                            "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

  performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

  with open(f'/content/drive/MyDrive/geotesouro/modelagem/models_save/convenios/mdl_ensem_convenio_{dict_orgaos[m]}.pkl', 'wb') as f:
      pickle.dump(clf, f, pickle.HIGHEST_PROTOCOL)

  X_preds = X_test
  X_preds["ano"] = "2022"

  preds_convenios = pd.DataFrame()
  preds_convenios = dataset[dataset["ano"]==2021][["codigo_ibge", "siafi_id", "nome"]].reset_index(drop=True)

  preds_convenios[[f"xgb_proba_0_{dict_orgaos[m]}", f"xgb_proba_1_{dict_orgaos[m]}"]] = mdl_xgb_convenio_res.predict_proba(X_preds)
  preds_convenios[[f"lgbm_proba_0_{dict_orgaos[m]}", f"lgbm_proba_1_{dict_orgaos[m]}"]] = mdl_lgbm_convenio_res.predict_proba(X_preds)
  preds_convenios[[f"cb_proba_0_{dict_orgaos[m]}", f"cb_proba_1_{dict_orgaos[m]}"]] = mdl_cb_convenio_res.predict_proba(X_preds)

  preds_convenios[[f"candido_pred_proba_0_{dict_orgaos[m]}", f"candido_pred_proba_1_{dict_orgaos[m]}"]] = clf.predict_proba(preds_convenios.drop(["codigo_ibge", "siafi_id", "nome"], 1))
  preds_convenios[f"candido_pred_{dict_orgaos[m]}"] = clf.predict(preds_convenios.drop(["codigo_ibge", "siafi_id", "nome", f"candido_pred_proba_0_{dict_orgaos[m]}", f"candido_pred_proba_1_{dict_orgaos[m]}"], 1))

  preds_conv = preds_conv.join(preds_convenios[[f"xgb_proba_0_{dict_orgaos[m]}", f"xgb_proba_1_{dict_orgaos[m]}", f"lgbm_proba_0_{dict_orgaos[m]}", f"lgbm_proba_1_{dict_orgaos[m]}",
                                              f"cb_proba_0_{dict_orgaos[m]}", f"cb_proba_1_{dict_orgaos[m]}", f"candido_pred_proba_0_{dict_orgaos[m]}", f"candido_pred_proba_1_{dict_orgaos[m]}",
                                              f"candido_pred_{dict_orgaos[m]}"]])

  0%|          | 0/11 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python

accuracy - min_desenv_reg = 0.5310592459605027
roc_auc - min_desenv_reg = 0.5850784252215513
log_loss -  min_desenv_reg = 0.8652150395002391


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:98: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- cb_proba_0_min_desenv_reg
- cb_proba_1_min_desenv_reg
- lgbm_proba_0_min_desenv_reg
- lgbm_proba_1_min_desenv_reg
- xgb_proba_0_min_desenv_reg
- ...
Feature names seen at fit time, yet now missing:
- cb_proba_0
- cb_proba_1
- lgbm_proba_0
- lgbm_proba_1
- xgb_proba_0
- ...

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWar

accuracy - min_turismo = 0.9215439856373429
roc_auc - min_turismo = 0.6577107265142043
log_loss -  min_turismo = 0.2574426271650146


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:98: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- cb_proba_0_min_turismo
- cb_proba_1_min_turismo
- lgbm_proba_0_min_turismo
- lgbm_proba_1_min_turismo
- xgb_proba_0_min_turismo
- ...
Feature names seen at fit time, yet now missing:
- cb_proba_0
- cb_proba_1
- lgbm_proba_0
- lgbm_proba_1
- xgb_proba_0
- ...

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The featu

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:98: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- cb_proba_0_min_educ
- cb_proba_1_min_educ
- lgbm_proba_0_min_educ
- lgbm_proba_1_min_educ
- xgb_proba_0_min_educ
- ...
Feature names seen at fit time, yet now missing:
- cb_proba_0
- cb_proba_1
- lgbm_proba_0
- lgbm_proba_1
- xgb_proba_0
- ...

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should

accuracy - min_cidad = 0.8590664272890485
roc_auc - min_cidad = 0.7207168518471402
log_loss -  min_cidad = 0.37466417220088566


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:98: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- cb_proba_0_min_cidad
- cb_proba_1_min_cidad
- lgbm_proba_0_min_cidad
- lgbm_proba_1_min_cidad
- xgb_proba_0_min_cidad
- ...
Feature names seen at fit time, yet now missing:
- cb_proba_0
- cb_proba_1
- lgbm_proba_0
- lgbm_proba_1
- xgb_proba_0
- ...

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names s

accuracy - min_def = 0.988689407540395
roc_auc - min_def = 0.9936675849403123
log_loss -  min_def = 0.06457785527162459


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:98: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- cb_proba_0_min_def
- cb_proba_1_min_def
- lgbm_proba_0_min_def
- lgbm_proba_1_min_def
- xgb_proba_0_min_def
- ...
Feature names seen at fit time, yet now missing:
- cb_proba_0
- cb_proba_1
- lgbm_proba_0
- lgbm_proba_1
- xgb_proba_0
- ...

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should matc

In [ ]:
preds_conv.to_sql('preds_convenios', con=conn, if_exists='replace', index=False)